# Problem:
Solve the kinematics of the target point in a `Scara` arm of 2 degree of freedom as show in the figure below.

![Scara arm - 2 DoF](https://raw.githubusercontent.com/Fernandohf/Robotics-Projects/master/media/scara.png)

## Reference Frames

The following reference frames where defined:
- `B0` is colinear with the base circle at arm heigh, simplifying the problem in 2D
- `B1` at the first joint, with relative rotation regarding `B0.z` defined by `theta_1`.
- `B2` at the second joint, with relative rotation regarding `B1.z` defined by `theta_2`.
  
Additionally, the first and second arm lengths were defined as `l_1`, `l_2`, respectively.

This image shows how the problems simplifies to 2D

!["Scara 2d simplification"](https://github.com/Fernandohf/Robotics-Projects/blob/master/media/evince_2021-10-24_09-46-43.png?raw=true)

In [1]:
# Funções das Bibliotecas Utilizadas
from sympy import symbols
from sympy.physics.mechanics import dynamicsymbols
from sympy.physics.vector import ReferenceFrame

# Variáveis Simbólicas
theta_1, theta_2 = dynamicsymbols('theta_1 theta_2')
l_1, l_2 = symbols('l_1 l_2', positive = True)

# Referenciais 
B0 = ReferenceFrame('B0')                 # Referencial Parado
B1 = ReferenceFrame('B1')     
B1.orient(B0, 'Axis', [theta_1, B0.z])    # Referencial móvel: theta_1 em relação a B0.z 
B2 = ReferenceFrame('B2')
B2.orient(B1, 'Axis', [theta_2, B1.z])    # Referencial móvel: theta_2 em relação a B1.z 

## Rotating Matrices
With all the reference frames correctly defined rotating matrices can be easily calculated.

In [2]:
# Matrizes de Rotação
T_B0B1 = B0.dcm(B1)    # Matriz de rotação de B1 para B0
T_B1B2 = B1.dcm(B2)    # Matriz de rotação de B2 para B1

T_B0B2 = (B0.dcm(B2)).simplify()     # Matriz de rotação de B0 para B3

# Resultados em LaTeX
from IPython.display import Latex
from sympy import latex

Latex("\\begin{eqnarray}" +
     "T_{\\theta_1}&=" + "&" +str(latex(T_B0B1)) + " T_{\\theta_2}=" + str(latex(T_B1B2)) + 
      "\\\\T_{\\theta_1, \\theta_2}&=" + "&" + str(latex(T_B0B2))    + "\\end{eqnarray}")

<IPython.core.display.Latex object>

## Points Relative Positions

Now, we can describe points $A$, $B$ and $C$ regarding the reference frames defined previously. 
- Point $A$ coincides with origin, therefore, $\vec{r_{OA}}=\vec{0}$
- Pointo $B$ can be expressed by reference frame `B1`, which $\vec{r_{AB}}={l_1 \vec{{i}_{B1}}}$
- Point $C$ is described by equation $\vec{r_{BC}}={l_2 \vec{{i}_{B2}}}$

Which $\vec{{i}_{B2}}$ and $\vec{{i}_{B3}}$ are vectors in direction $x$ of reference frame `B1` and `B2`, respectively. Following, it is noticeable how simply these vectors are defined using ```sympy```. The main advantage is that we do not have to worry about which frame the vectors are being defined, because all reference frames relations were already specified.

In [3]:
from sympy.physics.vector import Vector

# Vetores Posição entre os Pontos
r_OA = Vector(0)      # Vetor Nulo
r_AB = l_1 * B1.x     # Vetor que liga os pontos A e B expresso no referencial móvel B1
r_BC = l_2 * B2.x     # Vetor que liga os pontos B e C expresso no referencial móvel B2

R_AB = r_AB.express(B0) # Vetor que liga os pontos A e B expresso no referencial fixo B0
R_BC = r_BC.express(B0) # Vetor que liga os pontos B e C expresso no referencial fixo B0

# Resultado em LaTeX na Forma Vetorial ou Vetor Coluna
Latex("\\begin{eqnarray}" +
     "R_{AB}&=" + "&" +str(latex(R_AB)) + "\\\\" + 
     "R_{AB}&=" + "&" +str(latex(R_AB.to_matrix(B0))) +
     "\\end{eqnarray}")

<IPython.core.display.Latex object>

## Resulting matrices

Now, it is possible to solve kinematics of target point $C$ regarding reference frame `B0`. For that, position, velocity and acceleration vector needs to be solved for each point, $A$, $B$ and $C$. Additionally, some simplification is performed using ```trigsimp``` function.

In [4]:
from sympy.physics.vector import time_derivative, vlatex
from sympy import trigsimp

# Cinemática do ponto A em relação ao referencial B0
r_A  = r_OA.express(B0)
v_A = time_derivative(r_A, B0)
a_A = time_derivative(v_A, B0)

# Cinemática do ponto B em relação ao referencial B0
r_B  = r_A.express(B0) + r_AB.express(B0)
v_B = time_derivative(r_B, B0)
a_B = time_derivative(v_B, B0)

# Cinemática do ponto C em relação ao referencial B0
r_C  = r_B.express(B0) + r_BC.express(B0)
v_C = (time_derivative(r_C, B0))
a_C = (time_derivative(v_C, B0))

# Simplificação dos Resultados
r_A = (r_A.to_matrix(B0)).applyfunc(trigsimp)
v_A = (v_A.to_matrix(B0)).applyfunc(trigsimp)
a_A = (a_A.to_matrix(B0)).applyfunc(trigsimp)
r_B = (r_B.to_matrix(B0)).applyfunc(trigsimp)
v_B = (v_B.to_matrix(B0)).applyfunc(trigsimp)
a_B = (a_B.to_matrix(B0)).applyfunc(trigsimp)
r_C = (r_C.to_matrix(B0)).applyfunc(trigsimp)
v_C = (v_C.to_matrix(B0)).applyfunc(trigsimp)
a_C = (a_C.to_matrix(B0)).applyfunc(trigsimp)

# Resultados em LaTeX na forma de Vetores Coluna dos pontos A e B
Latex("\\begin{align}"
     "r_{A}&=" + str(vlatex(r_A)) + "&v_{A}&=" + str(vlatex(v_A)) + "&a_{A}&= " + str(vlatex(a_A)) + "\\" + "\\"
     "r_{B}&=" + str(vlatex(r_B)) + "&v_{B}&=" + str(vlatex(v_B)) + "&a_{B}&=" + str(vlatex(a_B)) + "\\" + "\\"
      "\\end{align}")

<IPython.core.display.Latex object>

In [5]:
# Resultados em LaTeX na forma de Vetores Coluna do ponto C
Latex("\\begin{align}"
     "\r_{C}=&" + str(vlatex(r_C)) + "\\\\" + "\\\\"
     "v_{C}=&" + str(vlatex(v_C)) + "\\\\" + "\\\\"
     "a_{C}=& " + str(vlatex(a_C)) + "\\end{align}")

<IPython.core.display.Latex object>